In [5]:
# ==============================================================================
# Notebook 3: Otimização e Interpretação de Modelos
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. Configuração e Carregamento de Bibliotecas e Dados
# ------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE

In [8]:
# --- Definindo os caminhos conforme a estrutura de diretórios ---
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

PROCESSED_DATA_PATH = os.path.join(ROOT_DIR, 'data', 'processed', 'df_final_processado.csv')
PROCESSED_SPLIT_PATH = os.path.join(ROOT_DIR, 'data', 'processed', 'split')
BASE_MODELS_PATH = os.path.join(ROOT_DIR, 'models', 'base_models')
FINAL_MODEL_PATH = os.path.join(ROOT_DIR, 'models', 'final_model')
REPORTS_PATH = os.path.join(ROOT_DIR, 'reports')

In [9]:
# Criando os diretórios se não existirem
os.makedirs(FINAL_MODEL_PATH, exist_ok=True)
os.makedirs(REPORTS_PATH, exist_ok=True)

# Carregando os conjuntos de treino e teste salvos no Notebook 2
print("Carregando conjuntos de treino e teste salvos...")
try:
    X_train = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'X_train.csv'))
    X_test = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'X_test.csv'))
    y_train = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'y_train.csv')).squeeze()
    y_test = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'y_test.csv')).squeeze()
    print("✅ Dados de treino e teste carregados com sucesso.")
except FileNotFoundError:
    print(f"❌ Erro: Arquivos de split não encontrados. Certifique-se de ter executado o Notebook 2.")
    X_train, X_test, y_train, y_test = None, None, None, None

# Carregando o DataFrame processado (para X e y)
try:
    df_processed = pd.read_csv(PROCESSED_DATA_PATH)
    print("✅ Dados processados carregados com sucesso.")
except FileNotFoundError:
    print(f"❌ Erro: Arquivo não encontrado em {PROCESSED_DATA_PATH}. Certifique-se de ter executado o Notebook 1.")
    df_processed = None

if df_processed is not None:
    # Divisão dos dados em X e y para o SMOTE
    X = df_processed.drop(columns=['Churn'])
    y = df_processed['Churn']

    # Carregando o melhor modelo base do Notebook 2 (vamos usar o Random Forest)
    try:
        model_name = 'random_forest'
        best_model = joblib.load(os.path.join(BASE_MODELS_PATH, f'{model_name}_base.pkl'))
        print(f"✅ Modelo '{model_name}_base' carregado com sucesso.")
    except FileNotFoundError:
        print(f"❌ Erro: Modelo '{model_name}_base.pkl' não encontrado. Certifique-se de ter executado o Notebook 2.")
        best_model = None

Carregando conjuntos de treino e teste salvos...
✅ Dados de treino e teste carregados com sucesso.
✅ Dados processados carregados com sucesso.
✅ Modelo 'random_forest_base' carregado com sucesso.


In [11]:
if best_model is not None:
    # ------------------------------------------------------------------------------
    # 2. Tratamento de Classes Desbalanceadas (SMOTE)
    # ------------------------------------------------------------------------------
    print("\nTratando o desbalanceamento de classes com SMOTE...")
    
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    print("✅ SMOTE aplicado. Dimensão do conjunto de treino após SMOTE:")
    print(f"X_train_res: {X_train_res.shape} | y_train_res: {y_train_res.shape}")


Tratando o desbalanceamento de classes com SMOTE...
✅ SMOTE aplicado. Dimensão do conjunto de treino após SMOTE:
X_train_res: (7228, 30) | y_train_res: (7228,)


In [12]:
# ------------------------------------------------------------------------------
# 3. Treinamento e Predição com o Modelo Otimizado
# ------------------------------------------------------------------------------
print("\nTreinando o modelo com dados balanceados...")
best_model.fit(X_train_res, y_train_res)
    
# Predições de probabilidade no conjunto de teste
y_pred_proba = best_model.predict_proba(X_test)[:, 1]


Treinando o modelo com dados balanceados...


In [14]:
# ------------------------------------------------------------------------------
# 4. Otimização do Limiar (Threshold)
# ------------------------------------------------------------------------------
print("\nOtimizando o limiar de decisão...")
    
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
    
f1_scores = [f1_score(y_test, y_pred_proba > t) for t in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)]
print(f"✅ Melhor limiar (threshold) encontrado (otimizando F1-Score): {best_threshold:.4f}")

y_pred_optimized = (y_pred_proba >= best_threshold).astype(int)


Otimizando o limiar de decisão...


NameError: name 'f1_score' is not defined

In [15]:
# ------------------------------------------------------------------------------
# 5. Avaliação Final do Modelo Otimizado
# ------------------------------------------------------------------------------
print("\n\n--- Avaliação do Modelo Otimizado ---")
print(confusion_matrix(y_test, y_pred_optimized))
print("\n" + classification_report(y_test, y_pred_optimized))
print(f"Área sob a curva (AUC): {auc:.4f}")

# Salvando o modelo final e os relatórios
joblib.dump(best_model, os.path.join(FINAL_MODEL_PATH, 'random_forest_final.pkl'))
report_path = os.path.join(REPORTS_PATH, 'relatorio_otimizacao.txt')
with open(report_path, 'w') as f:
    f.write("Relatório do Modelo Final Otimizado\n")
    f.write("-" * 35 + "\n")
    f.write(f"Melhor Threshold (F1-Score): {best_threshold:.4f}\n\n")
    f.write("Matriz de Confusão:\n")
    f.write(np.array2string(confusion_matrix(y_test, y_pred_optimized)) + "\n\n")
    f.write("Classification Report:\n")
    f.write(classification_report(y_test, y_pred_optimized))
print(f"✅ Modelo final salvo em '{os.path.join(FINAL_MODEL_PATH, 'random_forest_final.pkl')}'.")
print(f"✅ Relatório de otimização salvo em '{report_path}'.")



--- Avaliação do Modelo Otimizado ---


NameError: name 'y_pred_optimized' is not defined

In [16]:


# ------------------------------------------------------------------------------
# 6. Interpretação do Modelo e Feature Importance
# ------------------------------------------------------------------------------
print("\n--- Análise de Importância das Features ---")
if hasattr(best_model, 'feature_importances_'):
    feature_importances = best_model.feature_importances_
    feature_names = X.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)
        
    print(importance_df.head(10))
        
    # Salvando a importância das features
    importance_df.to_csv(os.path.join(REPORTS_PATH, 'feature_importance.csv'), index=False)
    print("✅ Relatório de importância das features salvo.")

print("\nProcesso do Notebook 3 concluído com sucesso!")


--- Análise de Importância das Features ---
                           Feature  Importance
4                           tenure    0.175106
8                     TotalCharges    0.173941
7                   MonthlyCharges    0.147675
28     InternetService_Fiber optic    0.053770
26  PaymentMethod_Electronic check    0.050929
24               Contract_Two year    0.043499
23               Contract_One year    0.025051
0                           gender    0.024884
2                          Partner    0.023852
3                       Dependents    0.022469
✅ Relatório de importância das features salvo.

Processo do Notebook 3 concluído com sucesso!
